# NFL Big Data Bowl

### How many yards will an NFL player gain after receiving a handoff?

Read the full data description on [here](https://www.kaggle.com/c/nfl-big-data-bowl-2020)

In [134]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import datetime
import math
import seaborn as sns

from string import punctuation

import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [135]:
train = pd.read_csv(r'train.csv')

C:\Users\jeffrey.tackes\Documents\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [136]:
train.head()

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,...,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW


# Data Dictionary

Each row in the file corresponds to a single player's involvement in a single play. The dataset was intentionally joined (i.e. denormalized) to make the API simple. All the columns are contained in one large dataframe which is grouped and provided by PlayId.

- GameId - a unique game identifier
- PlayId - a unique play identifier
- Team - home or away
- X - player position along the long axis of the field. See figure below.
- Y - player position along the short axis of the field. See figure below.
- S - speed in yards/second
- A - acceleration in yards/second^2
- Dis - distance traveled from prior time point, in yards
- Orientation - orientation of player (deg)
- Dir - angle of player motion (deg)
- NflId - a unique identifier of the player
- DisplayName - player's name
- JerseyNumber - jersey number
- Season - year of the season
- YardLine - the yard line of the line of scrimmage
- Quarter - game quarter (1-5, 5 == overtime)
- GameClock - time on the game clock
- PossessionTeam - team with possession
- Down - the down (1-4)
- Distance - yards needed for a first down
- c - which side of the field the play is happening on
- HomeScoreBeforePlay - home team score before play started
- VisitorScoreBeforePlay - visitor team score before play started
- NflIdRusher - the NflId of the rushing player
- OffenseFormation - offense formation
- OffensePersonnel - offensive team positional grouping
- DefendersInTheBox - number of defenders lined up near the line of scrimmage, spanning the width of the offensive line
- DefensePersonnel - defensive team positional grouping
- PlayDirection - direction the play is headed
- TimeHandoff - UTC time of the handoff
- TimeSnap - UTC time of the snap
- Yards - the yardage gained on the play (you are predicting this)
- PlayerHeight - player height (ft-in)
- PlayerWeight - player weight (lbs)
- PlayerBirthDate - birth date (mm/dd/yyyy)
- PlayerCollegeName - where the player attended college
- Position - the player's position (the specific role on the field that they typically play)
- HomeTeamAbbr - home team abbreviation
- VisitorTeamAbbr - visitor team abbreviation
- Week - week into the season
- Stadium - stadium where the game is being played
- Location - city where the game is being played
- StadiumType - description of the stadium environment
- Turf - description of the field surface
- GameWeather - description of the game weather
- Temperature - temperature (deg F)
- Humidity - humidity
- WindSpeed - wind speed in miles/hour
- WindDirection - wind direction

# Data Cleaning & Feature Engineering
The direction variable can be a bit confusion since it is the degree the player is moving relative to the play direction.  For this reason we will need to clean the variable such that the degree of player movement is representative of forward movement vs. backward movement.  We will adjust our variable so any angle between 0<x<180 represents forward movement, with 90 degree being straight down field, and 180 < x < 360 represents backward movement.

Lets adjust the PlayDirection variable to be 1 if the play is moving right and 0 if its moving left.

Also lets adjust the dir variable based on the direction of the play.

In [138]:

train['PlayDirection'] = train['PlayDirection'].apply(lambda x: x.strip() == 'right')

def new_orientation(angle, play_direction):
    if play_direction == 0:
        new_angle = 360.0 - angle
        if new_angle == 360.0:
            new_angle = 0.0
        return new_angle
    else:
        return angle

train['Orientation'] = train.apply(lambda row: new_orientation(row['Orientation'], row['PlayDirection']), axis=1)
train['Dir'] = train.apply(lambda row: new_orientation(row['Dir'], row['PlayDirection']), axis=1)    

In [139]:
# Lets see how many categorical columns we have and how we should handle those

cat_features = []

for col in train.columns:
    if train[col].dtype == 'object':
        cat_features.append((col, len(train[col].unique())))

cat_features

[('Team', 2),
 ('DisplayName', 2568),
 ('GameClock', 901),
 ('PossessionTeam', 32),
 ('FieldPosition', 33),
 ('OffenseFormation', 9),
 ('OffensePersonnel', 61),
 ('DefensePersonnel', 45),
 ('TimeHandoff', 30709),
 ('TimeSnap', 30721),
 ('PlayerHeight', 16),
 ('PlayerBirthDate', 1897),
 ('PlayerCollegeName', 314),
 ('Position', 25),
 ('HomeTeamAbbr', 32),
 ('VisitorTeamAbbr', 32),
 ('Stadium', 61),
 ('Location', 67),
 ('StadiumType', 34),
 ('Turf', 23),
 ('GameWeather', 74),
 ('WindSpeed', 70),
 ('WindDirection', 59)]

There are 23 categorical columns, but some of these look like they could be numerical, such as windspeed, height, time, gameclock.

In [140]:
train.columns

Index(['GameId', 'PlayId', 'Team', 'X', 'Y', 'S', 'A', 'Dis', 'Orientation',
       'Dir', 'NflId', 'DisplayName', 'JerseyNumber', 'Season', 'YardLine',
       'Quarter', 'GameClock', 'PossessionTeam', 'Down', 'Distance',
       'FieldPosition', 'HomeScoreBeforePlay', 'VisitorScoreBeforePlay',
       'NflIdRusher', 'OffenseFormation', 'OffensePersonnel',
       'DefendersInTheBox', 'DefensePersonnel', 'PlayDirection', 'TimeHandoff',
       'TimeSnap', 'Yards', 'PlayerHeight', 'PlayerWeight', 'PlayerBirthDate',
       'PlayerCollegeName', 'Position', 'HomeTeamAbbr', 'VisitorTeamAbbr',
       'Week', 'Stadium', 'Location', 'StadiumType', 'Turf', 'GameWeather',
       'Temperature', 'Humidity', 'WindSpeed', 'WindDirection'],
      dtype='object')

In [141]:
# Stadium Type

train.StadiumType.value_counts()

Outdoor                      362516
Outdoors                      92708
Indoors                       56826
Dome                          23122
Indoor                        19140
Retractable Roof              18766
Open                          11308
Retr. Roof-Closed             11044
Domed, closed                  6908
Retr. Roof - Closed            6446
Domed, open                    3696
Retr. Roof-Open                3014
Retractable Roof - Closed      2222
Closed Dome                    2134
Domed                          1826
Dome, closed                   1826
Domed, Open                    1760
OUTDOOR                        1254
Oudoor                         1188
indoor                         1166
Retr. Roof Closed              1056
Indoor, Roof Closed            1056
Bowl                            968
Outddors                        968
Heinz Field                     902
Outdoor Retr Roof-Open          880
Retr. Roof - Open               880
Indoor, Open Roof           

### Stadium
Some of the data above appears to have some misspellings.  We need to clean these up.

In [142]:

# Removes punctuation marks
def stadium_clean(stadium):
    if pd.isna(stadium):
        return np.nan
    stadium = stadium.lower() # make all text characters lowercase
    stadium = ''.join([c for c in stadium if c not in punctuation])
    stadium = stadium.replace('outside','outdoor')
    stadium = stadium.replace('ourdoor','outdoor')
    stadium = stadium.replace('outdor','outdoor')
    stadium = stadium.replace('oudoor','outdoor')
    stadium = stadium.replace('outddors','outdoor')  
    stadium = stadium.replace('outdoors','outdoor')
    stadium = stadium.replace('indoors','indoor')
    stadium = stadium.replace('retr roofopen','retractable roof open')
    stadium = stadium.replace('retr roof  open','retractable roof open')
    stadium = stadium.replace('outdoor retr roofopen','retractable roof open')
    stadium = stadium.replace('outdoor retractable roof open','retractable roof open')
    stadium = stadium.replace('indoor roof open','retractable roof open')
    stadium = stadium.replace('indoor open roof','retractable roof open')
    stadium = stadium.replace('dome open','retractable roof open')
    stadium = stadium.replace('retr roofclosed','retractable roof closed')
    stadium = stadium.replace('retr roof  closed','retractable roof closed')
    stadium = stadium.replace('retr roof closed','retractable roof closed')
    stadium = stadium.replace('indoor roof closed','retractable roof closed')
    stadium = stadium.replace('retractable roof  closed','retractable roof closed')
    stadium = stadium.replace('dome closed','retractable roof  closed')
    stadium = stadium.replace('domed closed','retractable roof  closed')
    stadium = stadium.replace('domed','dome')
    
    # We will focus on whether it is open/outdoors vs indoor/closed.
    if 'outdoor' in stadium or 'open' in stadium:
        return 1
    if 'indoor' in stadium or 'closed' in stadium:
        return 0
    
train['StadiumType'] = train["StadiumType"].apply(stadium_clean)

train.StadiumType.value_counts() 

1.0    484286
0.0    109824
Name: StadiumType, dtype: int64

### Turf
From a few trials, cleaning the turf variable shows a slight improvement in model performance.

In [143]:
grass_labels = ['grass', 'natural grass', 'natural', 'naturall grass']
train['Grass'] = np.where(train.Turf.str.lower().isin([grass_labels]), 1, 0)

### Possession
There are some issues with inconsistencies in team name abbreciations.  We will fix these now.

In [144]:
train[(train.PossessionTeam != train.HomeTeamAbbr) & (train.PossessionTeam != train.VisitorTeamAbbr)][['PossessionTeam','HomeTeamAbbr','VisitorTeamAbbr']].drop_duplicates()



,PossessionTeam,HomeTeamAbbr,VisitorTeamAbbr
2992,BLT,CIN,BAL
4334,CLV,CLE,PIT
5060,ARZ,DET,ARI
5962,HST,HOU,JAX
14872,HST,CIN,HOU
...,...,...,...
640134,BLT,BAL,NE
645062,CLV,CLE,BUF
648714,ARZ,TB,ARI
673222,CLV,CLE,MIA


In [145]:
train[(train.PossessionTeam != train.HomeTeamAbbr) & (train.PossessionTeam != train.VisitorTeamAbbr)][['PossessionTeam']].drop_duplicates()

# BLT = BAL
# CLV = CLE
# ARZ = ARI
# HST = HOU

,PossessionTeam
2992,BLT
4334,CLV
5060,ARZ
5962,HST


In [146]:
abbr_mapper = {'BAL':'BLT','CLE':'CLV','ARI':'ARZ','HOU':'HST'}
train['PossessionTeam'] = train['PossessionTeam'].map(abbr_mapper).fillna(train['PossessionTeam'])
train['HomeTeamAbbr'] = train['HomeTeamAbbr'].map(abbr_mapper).fillna(train['HomeTeamAbbr'])
train['VisitorTeamAbbr'] = train['VisitorTeamAbbr'].map(abbr_mapper).fillna(train['VisitorTeamAbbr'])


### Home Team Possession

In [147]:
train['HomeTeamPossession'] = np.where(train.PossessionTeam == train.HomeTeamAbbr, 1,0)

### Opponents Territory / Field Position
We will create a flag if the team is in the opponents territory.

In [148]:
train['OpponentsTerritory'] = np.where(train.PossessionTeam != train.FieldPosition,1,0)

### Offense Formation
Lets one-hot encode these variables.

In [149]:
train = pd.concat([train.drop(['OffenseFormation'], axis=1), pd.get_dummies(train['OffenseFormation'], prefix='Formation')], axis=1)
dummy_col = train.columns

In [150]:
train.head()

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,...,HomeTeamPossession,OpponenentsTerritory,Formation_ACE,Formation_EMPTY,Formation_I_FORM,Formation_JUMBO,Formation_PISTOL,Formation_SHOTGUN,Formation_SINGLEBACK,Formation_WILDCAT
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,278.01,182.82,...,1,0,0,0,0,0,0,1,0,0
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,332.39,161.30,...,1,0,0,0,0,0,0,1,0,0
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,356.99,157.27,...,1,0,0,0,0,0,0,1,0,0
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,0.23,254.36,...,1,0,0,0,0,0,0,1,0,0
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,347.37,195.69,...,1,0,0,0,0,0,0,1,0,0


## Game Clock



In [151]:
train['GameClock'].value_counts()

15:00:00    19008
02:00:00     7172
14:54:00     2794
14:55:00     2640
14:56:00     1694
            ...  
14:38:00      154
14:58:00      110
14:39:00       88
14:59:00       88
00:00:00       22
Name: GameClock, Length: 901, dtype: int64

In [152]:
def strtoseconds(txt):
    txt = txt.split(':')
    ans = int(txt[0])*60 + int(txt[1]) + int(txt[2])/60
    return ans

train['GameClock'] = train['GameClock'].apply(strtoseconds)


### Player Height, Birthdate, Time at Handoff, Time at snap



In [153]:

train['PlayerHeight'] = train['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))
train['TimeHandoff'] = train['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
train['TimeSnap'] = train['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
train['TimeDelta'] = train.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
train['PlayerBirthDate'] = train['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))
seconds_in_year = 60*60*24*365.25
train['PlayerAge'] = train.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)
train = train.drop(['TimeHandoff', 'TimeSnap', 'PlayerBirthDate'], axis=1)

### Wind Speed and Direction

### Weather

We are going to apply the following preprocessing:

- Lower case
- N/A Indoor, N/A (Indoors) and Indoor => indoor Let's try to cluster those together.
- coudy and clouidy => cloudy
- party => partly
- sunny and clear => clear and sunny
- skies and mostly => ""



To encode our weather we are going to do the following map:

climate controlled or indoor => 3, sunny or sun => 2, clear => 1, cloudy => -1, rain => -2, snow => -3, others => 0
partly => multiply by 0.5
I don't have any expercience with american football so I don't know if playing in a climate controlled or indoor stadium is good or not, if someone has a good idea on how to encode this it would be nice to leave it in the comments :)

In [154]:
train.GameWeather.value_counts()

Cloudy           147114
Sunny            143088
Partly Cloudy     55880
Clear             55726
Mostly Cloudy     27962
                  ...  
Party Cloudy        814
Light rain          792
Sunny, Windy        792
Partly clear        770
Rainy               704
Name: GameWeather, Length: 73, dtype: int64

In [155]:
train['GameWeather'] = train['GameWeather'].str.lower()
indoor = "indoor"
train['GameWeather'] = train['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
train['GameWeather'] = train['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
train['GameWeather'] = train['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
train['GameWeather'] = train['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)

In [156]:
def map_weather(txt):
    ans = 1
    if pd.isna(txt):
        return 0
    if 'partly' in txt:
        ans*=0.5
    if 'climate controlled' in txt or 'indoor' in txt:
        return ans*3
    if 'sunny' in txt or 'sun' in txt:
        return ans*2
    if 'clear' in txt:
        return ans
    if 'cloudy' in txt:
        return -ans
    if 'rain' in txt or 'rainy' in txt:
        return -2*ans
    if 'snow' in txt:
        return -3*ans
    return 0

In [157]:
train['GameWeather'] = train['GameWeather'].apply(map_weather)

### RusherID flag

In [158]:
train['IsRusher'] = np.where(train.NflId == train.NflIdRusher,1,0)
#train.drop(['NflId','NflIdRusher'], axis = 1, inplace = True)

#### Same Fields
- GameId
- PlayId
- Season
- YardLine
- Quarter
- GameClock
- Down
- Distance
- FieldPosition
- HomeScoreBeforePlay
- VisitorScoreBeforePlay
- NflIdRusher
- OffensePersonnel
- DefendersInTheBox
- DefensePersonnel
- PlayDirection
- Yards
- HomeTeamAbbr
- VisitorTeamAbbr
- Week
- Stadium
- Location
- StatiumType
- Turf
- GameWeather
- Temperature
- Humidity
- WindSpeed
- WindDirection
- Grass
- HomeTeamPossession
- OpponenentsTerritory
- Formation_ACE
- Formation_EMPTY
- Formation_I_FORM
- Formation_JUMBO
- Formation_PISTOL
- Formation_SHOTGUN
- Formation_SINGLEBACK
- Formation_WILDCAT
- TimeDelta
- 

#### Different fields
- Team
- X
- Y
- S
- A
- Orientation
- Dir
- NflId
- DisplayName
- JerseyNumber
- PossessionTeam
- PlayerHeight
- PlayerWeight
- PlayerCollegeName
- Position
- PlayerAge
- IsRusher

In [159]:
#train[train.PlayId ==20170907000118].iloc[:,55:65]
train.columns

Index(['GameId', 'PlayId', 'Team', 'X', 'Y', 'S', 'A', 'Dis', 'Orientation',
       'Dir', 'NflId', 'DisplayName', 'JerseyNumber', 'Season', 'YardLine',
       'Quarter', 'GameClock', 'PossessionTeam', 'Down', 'Distance',
       'FieldPosition', 'HomeScoreBeforePlay', 'VisitorScoreBeforePlay',
       'NflIdRusher', 'OffensePersonnel', 'DefendersInTheBox',
       'DefensePersonnel', 'PlayDirection', 'Yards', 'PlayerHeight',
       'PlayerWeight', 'PlayerCollegeName', 'Position', 'HomeTeamAbbr',
       'VisitorTeamAbbr', 'Week', 'Stadium', 'Location', 'StadiumType', 'Turf',
       'GameWeather', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection',
       'Grass', 'HomeTeamPossession', 'OpponenentsTerritory', 'Formation_ACE',
       'Formation_EMPTY', 'Formation_I_FORM', 'Formation_JUMBO',
       'Formation_PISTOL', 'Formation_SHOTGUN', 'Formation_SINGLEBACK',
       'Formation_WILDCAT', 'TimeDelta', 'PlayerAge', 'IsRusher'],
      dtype='object')

In [160]:
rusher_coords = train[train.IsRusher==1][['PlayId','X','Y']]
rusher_coords.rename(columns = {'X':'Rusher_X','Y':'Rusher_Y'}, inplace = True)
train = pd.merge(train, rusher_coords, how = 'left', on = 'PlayId')

In [161]:
distance_from_rusher(2,4,7,9)

7.0710678118654755

In [210]:
def distance_from_rusher(player_x, player_y, rusher_x, rusher_y):
    #player_x.astype(float)
    
    DistanceFromRusher = math.sqrt((player_x-rusher_x)**2 + (player_y-rusher_y)**2) + np.random.normal(0,.001,1)[0]
    return DistanceFromRusher

train['DistanceFromRusher'] = train.apply(lambda x: distance_from_rusher(x.X, x.Y, x.Rusher_X, x.Rusher_Y), axis=1)




In [209]:
np.random.normal(0,.001,1)[0]

0.0019194212500862158

In [212]:
train['Rank'] = train.groupby(['PlayId','Team']).DistanceFromRusher.rank('dense', ascending = True)
train['TeamRank']= train.Team + train.Rank.astype(str)
train.head()

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,...,Formation_SINGLEBACK,Formation_WILDCAT,TimeDelta,PlayerAge,IsRusher,Rusher_X,Rusher_Y,DistanceFromRusher,Rank,TeamRank
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,278.01,182.82,...,0,0,1.0,28.692760,0,78.75,30.53,6.480255,4.0,away4.0
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,332.39,161.30,...,0,0,1.0,28.457305,0,78.75,30.53,4.593143,1.0,away1.0
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,356.99,157.27,...,0,0,1.0,28.629790,0,78.75,30.53,5.447795,3.0,away3.0
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,0.23,254.36,...,0,0,1.0,34.795430,0,78.75,30.53,7.819427,6.0,away6.0
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,347.37,195.69,...,0,0,1.0,30.061685,0,78.75,30.53,10.621396,8.0,away8.0


In [213]:
train_melt = pd.melt(train, id_vars = ['PlayId','TeamRank'], value_vars = ['X','Y','S','A','Orientation','Dir'])
train_melt['NewColumnName'] =  train_melt.TeamRank + train_melt.variable
train_melt = train_melt[['PlayId','NewColumnName','value']]

In [226]:
#train_melt[train_melt.duplicated()==True]
#train_melt = train_melt.set_index('PlayId')
train_melt = train_melt.pivot(index = 'PlayId', columns = 'NewColumnName',values = 'value')


KeyError: 'PlayId'

In [227]:
train_melt = train_melt.reset_index()

In [250]:
# Removed OffensePersonnel - Might want to create columns for the various types of offense quantities
train_same = train[train.IsRusher==1][[
    'PlayId', 'Season','YardLine','Quarter','GameClock','Down','Distance','HomeScoreBeforePlay','VisitorScoreBeforePlay',
    'DefendersInTheBox','PlayDirection','Week','StadiumType','Grass','HomeTeamPossession','OpponentsTerritory',
    'Formation_ACE','Formation_EMPTY','Formation_I_FORM','Formation_JUMBO','Formation_PISTOL','Formation_SHOTGUN',
    'Formation_SINGLEBACK','Formation_WILDCAT','TimeDelta','Yards']]

    

In [251]:
train_final = pd.merge(train_same, train_melt, on = 'PlayId',how = 'left')

In [252]:
# Make sure all categorical variables are gone

cat_features = []

for col in train_final.columns:
    if train_final[col].dtype=='object':
        cat_features.append(col)
        
cat_features

[]

### Yards Left
We will calculate how many yards left to go to the endzone.  We will also need to clean up some data issues where yards left < yards.

In [26]:
train['HomeField'] = train['FieldPosition'] == train['HomeTeamAbbr']
train['YardsLeft'] = train.apply(lambda row: 100-row['YardLine'] if row['HomeField'] else row['YardLine'], axis=1)
train['YardsLeft'] = train.apply(lambda row: row['YardsLeft'] if row['PlayDirection'] else 100-row['YardsLeft'], axis=1)
train.drop(train.index[(train['YardsLeft']<train['Yards']) | (train['YardsLeft']-100>train['Yards'])], inplace=True)

# Split to Train & Test

In [254]:
train_df = train_final[train_final.Season ==2019]
test_df = train_final[train_final.Season !=2019]

# Remove Target Variable

In [255]:
y_train = train_df.pop('Yards')
y_test = test_df.pop('Yards')

# Change to TensorFlow dataset

In [260]:
# Ensure everything is float
train_df = np.asarray(train_df).astype(np.float32)

tf_df = tf.data.Dataset.from_tensor_slices((train_df, y_train))

<TensorSliceDataset shapes: ((157,), ()), types: (tf.float32, tf.int32)>

# Normalize all data

# Build Model

In [263]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [264]:
model = build_model()

EPOCHS = 1000

history = model.fit(
  tf_df, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])

Epoch 1/15
     55/Unknown - 1s 11ms/step - loss: nan - accuracy: 0.1319

C:\Users\jeffrey.tackes\Documents\Anaconda3\lib\site-packages\tensorflow_core\python\keras\utils\generic_utils.py:441: RuntimeWarning: invalid value encountered in greater
  if abs(avg) > 1e-3:


7836/7836 [==============================] - 26s 3ms/step - loss: nan - accuracy: 0.0945
Epoch 2/15
7836/7836 [==============================] - 8s 1ms/step - loss: nan - accuracy: 0.0946
Epoch 3/15
7836/7836 [==============================] - 8s 1ms/step - loss: nan - accuracy: 0.0946
Epoch 4/15
7836/7836 [==============================] - 8s 1ms/step - loss: nan - accuracy: 0.0946
Epoch 5/15
7836/7836 [==============================] - 9s 1ms/step - loss: nan - accuracy: 0.0946
Epoch 6/15
7836/7836 [==============================] - 10s 1ms/step - loss: nan - accuracy: 0.0946
Epoch 7/15
7836/7836 [==============================] - 8s 1ms/step - loss: nan - accuracy: 0.0946
Epoch 8/15
7836/7836 [==============================] - 8s 1ms/step - loss: nan - accuracy: 0.0946
Epoch 9/15
7836/7836 [==============================] - 8s 1ms/step - loss: nan - accuracy: 0.0946
Epoch 10/15
7836/7836 [==============================] - 8s 1ms/step - loss: nan - accuracy: 0.0946
Epoch 11/15
7836/7

In [27]:
# First pass lets use non categorical variables

train_baseline = train.sort_values(by =['PlayId','Team','JerseyNumber']).reset_index()
train_baseline.drop(['GameId','PlayId','Team','index'], axis=1, inplace = True)


In [28]:
# For the baseline model we will drop all the categorical features.

cat_features = []

for col in train_baseline.columns:
    if train[col].dtype=='object':
        cat_features.append(col)

train_baseline = train_baseline.drop(cat_features, axis = 1)

In [29]:
# Unpivot all the rows to 1 row for each play

train_baseline.fillna(-999, inplace = True)

players_col = []
for col in train_baseline.columns:
    if train_baseline[col][:22].std()!=0:
        players_col.append(col)
        
        

In [38]:
train.NflId.unique()

AttributeError: 'DataFrame' object has no attribute 'NflId'

In [30]:
X_train = np.array(train_baseline[players_col]).reshape(-1, len(players_col)*22)

In [150]:
play_col = train.drop(players_col+['Yards'], axis=1).columns
X_play_col = np.zeros(shape=(X_train.shape[0], len(play_col)))
for i, col in enumerate(play_col):
    X_play_col[:, i] = train[col][::22]

ValueError: could not convert string to float: 'away'